In [16]:
# !pip install tokenizers --user
!pip install --user -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from tokenizers import Tokenizer
from tokenizers import trainers
from tokenizers.normalizers import StripAccents, Lowercase, Sequence
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer, UnigramTrainer
from tokenizers.models import BPE, Unigram

from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
import tokenizers

import re
#from hazm import *
%load_ext autoreload
%autoreload 2

In [6]:
#! pip install tokenizers scikit-learn


In [7]:
#! pip install hazm


In [8]:
#! pip install tokenizers scikit-learn
#! pip install hazm



In [9]:
from src.helper import get_lines_without_number
from src.helper import regex_clean
from src.helper import clean_spanish_text
import re

In [10]:
## Load the Spanish text 
spanish_text_path = "content/spa_wikipedia_2021_30K-sentences.txt"

final_L = get_lines_without_number(spanish_text_path)
#Get the data as lines 
# Already cleaned from starting numbers
whole_text = get_lines_without_number(spanish_text_path)
#Use the cleaning regex
whole_text = clean_spanish_text(whole_text)
result_string = ""
for x in whole_text: result_string += x


In [11]:
result_string[:500]

' 12 de abril de 1996 es una actriz de la industria del entretenimiento para adultos y una personalidad de internet. 15 Y el Señor dijo a Noé La hijas de tus hijos se han vendido por lo que he aquí mi ira está encendida en contra de los hijos de los hombres porque no quieren escuchar mi voz.  1948  1958 los grupos paramilitares del conservatismo inician una persecución contra los campesinos liberales y comunistas.  1954 conocido simplemente como Vacca es un artista español.  1957 después de que e'

In [12]:
# This cell encodes the whole text with the tokenizer 

tokenizer =  Tokenizer.from_file("model/spanish_tok_model.json")
tokenized_text = []
#Use this line to check if Tokinzier works. Schould be three element list 
#print(tokenizer.encode("nosotros saltamos").tokens)
for sent in whole_text:
    #break
    encoded_sent = tokenizer.encode(sent).tokens
    tokenized_text.append(encoded_sent)
    
    
    #print(encoded_sent)
    


In [13]:
train_corpus, temp_corpus = train_test_split(tokenized_text, test_size=0.2, random_state=42)
val_corpus, test_corpus = train_test_split(temp_corpus, test_size=0.5, random_state=42)

# Example
print("Training Data Example:", train_corpus[0])
print("Validation Data Example:", val_corpus[0])
print("Test Data Example:", test_corpus[0])

Training Data Example: ['mide', '16', 'rasgos', 'de', 'primer', 'orden', 'que', 'dan', 'una', 'información', 'rica', 'y', 'detallada', 'de', 'la', 'personalidad', 'del', 'sujeto', 'y', 'cinco', 'dimensiones', 'globales', 'las', 'cuales', 'narran', 'la', 'relación', 'de', 'los', 'rasgos', 'de', 'primer', 'orden', '.']
Validation Data Example: ['todos', 'estos', 'hermanos', 'buscan', 'favorecer', 'la', 'comunión', 'y', 'la', 'interdependencia', 'en', 'el', 'seno', 'del', 'instituto', '.']
Test Data Example: ['la', 'especie', 'tipo', 'w', 'sato', 'i', 'fue', 'descrita', 'por', 'o', 'kaza', 'ki', 'en', '1992', 'a', 'partir', 'de', 'un', 'único', 'diente', '.']


In [17]:
from nltk import word_tokenize
from nltk.lm import MLE
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm.preprocessing import padded_everygram_pipeline

# Set the n-gram size
n = 3

# Check pre-processing on a subset:

#This preprocessing step helps models capture context effectively, especially at sentence boundaries.

ngram_data, padded = padded_everygram_pipeline(n, tokenized_text[0:10])

# What is the effect of padding?

# Padding adds special tokens (start symbol <s>) and (end symbol </s>) to the text to signify sentence boundaries.
# This ensures that n-grams near the edges of a sentence (e.g., start or end) still form complete n-grams
# by including these tokens. For example, in trigrams, first_word and last_word are generated.

print("PADDING:")
print(list(padded))

# Which ngrams do we get?

# Unigrams: Individual words or tokens, e.g., ('<s>',), ('volume',).
# Bigrams: Pairs of tokens, e.g., ('<s>', '<s>'), ('<s>', 'volume').
# Trigrams: Groups of three tokens, e.g., ('<s>', '<s>', 'volume'), ('<s>', 'volume', 'i').
# Padding ensures valid n-grams even at sentence start/end.

print("\n\nNGRAMS:")
for ngrams in ngram_data:
    print(list(ngrams))
    print()

PADDING:
['<s>', '<s>', '12', 'de', 'abril', 'de', '1996', 'es', 'una', 'actriz', 'de', 'la', 'industria', 'del', 'entretenimiento', 'para', 'adultos', 'y', 'una', 'personalidad', 'de', 'internet', '.', '</s>', '</s>', '<s>', '<s>', '15', 'y', 'el', 'señor', 'dijo', 'a', 'noé', 'la', 'hijas', 'de', 'tus', 'hijos', 'se', 'han', 'vendido', 'por', 'lo', 'que', 'he', 'aquí', 'mi', 'ira', 'está', 'encen', 'dida', 'en', 'contra', 'de', 'los', 'hijos', 'de', 'los', 'hombres', 'porque', 'no', 'quieren', 'escuchar', 'mi', 'voz', '.', '</s>', '</s>', '<s>', '<s>', '1948', '1958', 'los', 'grupos', 'paramilitares', 'del', 'conserva', 'tismo', 'inician', 'una', 'persecución', 'contra', 'los', 'campesinos', 'liberales', 'y', 'comunistas', '.', '</s>', '</s>', '<s>', '<s>', '1954', 'conocido', 'simplemente', 'como', 'vac', 'ca', 'es', 'un', 'artista', 'español', '.', '</s>', '</s>', '<s>', '<s>', '1957', 'después', 'de', 'que', 'el', 'tempera', 'mental', 'fre', 'da', 'también', 'hubiera', 'salido', '

In [18]:
from nltk.lm import MLE

n=3

# Train data is an iterator over the pre-processed input
train_data, padded_sents = padded_everygram_pipeline(n, train_corpus)

model = MLE(n)
model.fit(train_data, padded_sents)

In [19]:
# Let's have a look at the vocabulary
print(model.vocab)
print()
print(model.vocab.counts.most_common(50))
print()
print(model.vocab.counts.most_common()[-50:])

# You can see that the text is much cleaner than the corpora that were crawled from the web

<Vocabulary with cutoff=1 unk_label='<UNK>' and 27887 items>

[('<s>', 48000), ('</s>', 48000), ('de', 36911), ('.', 24000), ('la', 20485), ('en', 16882), ('el', 16152), ('y', 13110), ('a', 10480), ('que', 10185), ('los', 7906), ('se', 7568), ('del', 6166), ('un', 5539), ('su', 5286), ('con', 5123), ('por', 5062), ('las', 5042), ('una', 5017), ('para', 3733), ('es', 3411), ('al', 2970), ('como', 2826), ('fue', 2607), ('no', 2385), ('más', 2096), ('lo', 1905), ('sus', 1798), ('o', 1464), ('también', 1288), ('le', 1138), ('este', 1137), ('entre', 1067), ('pero', 1025), ('dos', 1000), ('ha', 960), ('esta', 959), ('sin', 946), ('años', 866), ('son', 854), ('ser', 818), ('desde', 777), ('sobre', 771), ('hasta', 761), ('durante', 744), ('cuando', 736), ('era', 735), ('después', 722), ('e', 676), ('año', 658)]

[('trav', 1), ('íen', 1), ('ork', 1), ('entaron', 1), ('billie', 1), ('novedosa', 1), ('elgin', 1), ('indefinida', 1), ('precandidato', 1), ('deus', 1), ('compag', 1), ('pregrado', 1),

In [20]:
print(model.counts)
# counts for unigrams:
print(model.counts['el']) # i.e. Count('not')
# count for bigrams
print(model.counts[['y']]['el']) # i.e. Count('not'|'was')
# count for trigrams
print(model.counts[["una","de"]]["las"]) # i.e.'una', 'de', 'las'

<NgramCounter with 3 ngram orders and 1927440 ngrams>
16152
745
174


In [21]:
all_tokens = [tok for sent in train_corpus for tok in sent]
num_tokens = len(all_tokens)
num_sentences = len(train_corpus)

model_score = model.score('no')
probability = model.counts['no']/num_tokens


print("\nProbability of the word 'no'")
print("{:.5f}".format(model_score))
print("{:.5f}".format(probability))

print("\nAdjust for padding tokens")
all_padding_tokens = num_sentences * (n-1) * 2
print(num_tokens, all_padding_tokens)

adjusted_probability = model.counts['no']/(num_tokens + all_padding_tokens)
print("{:.5f}".format(adjusted_probability))

print("\nProbabilities padding tokens")
print("{:.5f}".format(model.score('<s>')))
print("{:.5f}".format(model.score('</s>')))


Probability of the word 'no'
0.00358
0.00418

Adjust for padding tokens
570480 96000
0.00358

Probabilities padding tokens
0.07202
0.07202


In [22]:
# We can also calculate the frequency for higher n-grams.
# bigram
print(model.score('una', ['de']))  # P('not'|'is')
# trigram
print(model.score('una', ['los', 'de']))  # P('not'|'emma is')
print()

0.014819430522066593
0.01694915254237288



In [23]:
# To avoid underflow when working with many small score values, we usually work with log probabilities instead.
# This can be done with the `logscore` method.
print(model.score('una', ['de']))
print(model.logscore('una', ['los', 'de']))   #Todos los quesos #de mayo de ##'una', 'de', 'las'

0.014819430522066593
-5.882643049361842


In [24]:
# The vocabulary helps us handle words that have not occurred during training.
# If we lookup the vocab on unseen sentences not from the training data,
# it automatically replace words not in the vocabulary with `<UNK>`.
sent = "El volumen de la undidad C es OS/2."
print(model.vocab.lookup(tokenizer.encode(sent).tokens))

('el', 'volumen', 'de', 'la', 'undidad', 'c', 'es', 'os', '<UNK>', '2', '.')


In [25]:
# Items that are not seen during training are mapped to the vocabulary's "unknown label" token.  This is "<UNK>" by default.
print(model.score("<UNK>") == model.score("OS/"))

# The MLE model does not apply any smoothing, so the probability for UNK is 0
print(model.score("<UNK>"),model.logscore("<UNK>") )

# As a consequence, the probability for a phrase containing an unknown word is also 0.
print(model.score("es",["OS/","os"]), model.logscore("es",["OS/","os"]))

True
0.0 -inf
0 -inf


In [26]:
from nltk.lm import Laplace
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, train_corpus)
smoothed_model_small =  Laplace(n)
smoothed_model_small.fit(train_data, padded_sents)

In [27]:
x = "OS/"
longXC = ("es",["OS/","os"])
print(smoothed_model_small.score(x))
print(smoothed_model_small.score(x))
print(smoothed_model_small.score("es",["OS/","os"]))
print()
print(smoothed_model_small.logscore(x))
print(smoothed_model_small.logscore(x))
print(smoothed_model_small.logscore(longXC))

1.4401606067108604e-06
1.4401606067108604e-06
3.585900240255316e-05

-19.40533885933377
-19.40533885933377
-19.40533885933377


In [28]:
smoothed_model_small.generate(text_seed=tokenizer.encode("A los artistas se les").tokens, num_words=15, random_seed=1)

['contabili',
 'za',
 'varias',
 'veces',
 'hasta',
 'que',
 'murieron',
 'entre',
 '1000',
 'y',
 'tx',
 '2000',
 'para',
 'desarrollar',
 'las']

In [29]:
test_data, _ = padded_everygram_pipeline(n, val_corpus)

perplexity = []
for test in test_data:
  perplexity.append(smoothed_model_small.perplexity(test))

In [30]:
values = []
#TODO
for i in range(len(perplexity)):
  if not np.isinf(perplexity[i]):
    values.append(i)
  else:
      pass
print(f"{len(values)}/{len(perplexity)}")

3000/3000


In [31]:
valid_perplexity = [perplexity[i] for i in values]
idx = np.argpartition(valid_perplexity, 10)
# vali1d_perplexity[idx[0]]

In [32]:
print(f"length of the whole test data: {len(perplexity)}")
print(f"length of the filtered test data: {len(valid_perplexity)}")

length of the whole test data: 3000
length of the filtered test data: 3000


In [33]:
for i in values:
  print("({0}):{1}".format(val_corpus[i], perplexity[i]))

(['todos', 'estos', 'hermanos', 'buscan', 'favorecer', 'la', 'comunión', 'y', 'la', 'interdependencia', 'en', 'el', 'seno', 'del', 'instituto', '.']):2083.1957529256397
(['durante', 'unos', 'años', 'los', 'tá', 'pers', 'de', 'plástico', 'domin', 'aron', 'el', 'imperio', 'de', 'las', 'fi', 'amb', 'r', 'eras', '.']):3148.2162418528974
(['además', 'como', 'congresista', 'impulsó', 'varias', 'leyes', 'en', 'favor', 'de', 'la', 'niñez', 'y', 'la', 'mujer', 'los', 'derechos', 'de', 'los', 'adolescentes', 'y', 'en', 'contra', 'de', 'la', 'violencia', 'familiar', '.']):1509.8518574088619
(['el', 'one', 'ar', 'med', 'ban', 'dit', 'el', 'hospital', 'general', 'carson', 'la', 'rock', 'ford', 'station', 'el', 'túnel', 'porter', 'y', 'el', 'ferry', 'de', 'liberty', 'city', 'son', 'los', 'lugares', 'más', 'famosos', 'del', 'lugar', '.']):4617.865905342526
(['debido', 'a', 'estas', 'escu', 'chas', 'el', 'álbum', 'se', 'logró', 'fil', 'trar', 'por', 'completo', 'en', 'internet', 'tres', 'días', 'antes

# Next 

In [50]:
import tiktoken

In [51]:
tokenizer = tiktoken.get_encoding("gpt2")

In [52]:
with open(spanish_text_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 3924734
1	; 12 de abril de 1996) es una actriz de la industria del entretenimiento para adultos y una perso


In [54]:
with open(spanish_text_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

# Remove newlines and duplicated whitespaces
raw_text = raw_text.replace("\n", " ")
raw_text = raw_text.replace("  ", " ")
enc_text = tokenizer.encode(raw_text)

In [55]:
print(raw_text[0:100])
print()
print(tokenizer.decode(enc_text[0:50]))
print()
tokens = [tokenizer.decode([tok]) for tok in enc_text[0:50]]
print(enc_text[0:50])
print()
print(tokens)

1	; 12 de abril de 1996) es una actriz de la industria del entretenimiento para adultos y una person

1	; 12 de abril de 1996) es una actriz de la industria del entretenimiento para adultos y una personalidad de internet. 2	"15 Y el Señor dijo a No

[16, 197, 26, 1105, 390, 450, 22379, 390, 8235, 8, 1658, 555, 64, 719, 47847, 390, 8591, 2226, 7496, 1619, 920, 1186, 268, 320, 1153, 78, 31215, 4044, 418, 331, 555, 64, 2614, 32482, 390, 5230, 13, 362, 197, 1, 1314, 575, 1288, 1001, 12654, 273, 2566, 7639, 257, 1400]

['1', '\t', ';', ' 12', ' de', ' ab', 'ril', ' de', ' 1996', ')', ' es', ' un', 'a', ' act', 'riz', ' de', ' la', ' indust', 'ria', ' del', ' ent', 'ret', 'en', 'im', 'ient', 'o', ' para', ' adult', 'os', ' y', ' un', 'a', ' personal', 'idad', ' de', ' internet', '.', ' 2', '\t', '"', '15', ' Y', ' el', ' Se', 'ñ', 'or', ' di', 'jo', ' a', ' No']


In [56]:
enc_sample = train_corpus[7:99]

context_length = 4
for i in range(1, context_length+1):
    context = enc_sample[:i]
    target = enc_sample[i]

    print(context, "-->", target)
    # print(tokenizer.decode(context), "-->",tokenizer.decode([target]))

[['tienen', 'autonomía', 'financiera', 'y', 'disponen', 'de', 'su', 'propio', 'presupuesto', 'que', 'administra', 'y', 'reparte', 'el', 'consejo', 'regional', '.']] --> ['ha', 'publicado', 'varios', 'libros', 'sobre', 'la', 'botánica', 'de', 'las', 'islas', 'del', 'pacífico', 'entre', 'ellos', 'la', 'polines', 'ia', 'medicina', 'her', 'bal', '1992', 'y', 'ha', 'escrito', 'numerosos', 'artículos', 'científicos', 'sobre', 'plantas', 'medicinales', 'etno', 'botánica', 'y', 'flor', 'ística', 'de', 'la', 'polines', 'ia', '.']
[['tienen', 'autonomía', 'financiera', 'y', 'disponen', 'de', 'su', 'propio', 'presupuesto', 'que', 'administra', 'y', 'reparte', 'el', 'consejo', 'regional', '.'], ['ha', 'publicado', 'varios', 'libros', 'sobre', 'la', 'botánica', 'de', 'las', 'islas', 'del', 'pacífico', 'entre', 'ellos', 'la', 'polines', 'ia', 'medicina', 'her', 'bal', '1992', 'y', 'ha', 'escrito', 'numerosos', 'artículos', 'científicos', 'sobre', 'plantas', 'medicinales', 'etno', 'botánica', 'y', 'f

In [57]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, context_length):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - context_length):
            input_sequence = token_ids[i:i + context_length]

            #shift to the right
            target_sequence = token_ids[i + 1: i + context_length + 1]

            # input and output are represented as tensors
            self.input_ids.append(torch.tensor(input_sequence))
            self.target_ids.append(torch.tensor(target_sequence))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [58]:
def create_dataloader(txt, batch_size=8, context_length=4, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataset(txt, tokenizer, context_length)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [59]:
# Set a manual seed for reproducibility of shuffling and weight initialization
torch.manual_seed(0)
dataloader = create_dataloader(raw_text, batch_size=8, context_length=4, shuffle=False)

In [60]:
print(len(dataloader))
data_iter = iter(dataloader)

first_batch = next(data_iter)
print("1:", first_batch)

170978
1: [tensor([[   16,   197,    26,  1105],
        [  197,    26,  1105,   390],
        [   26,  1105,   390,   450],
        [ 1105,   390,   450, 22379],
        [  390,   450, 22379,   390],
        [  450, 22379,   390,  8235],
        [22379,   390,  8235,     8],
        [  390,  8235,     8,  1658]]), tensor([[  197,    26,  1105,   390],
        [   26,  1105,   390,   450],
        [ 1105,   390,   450, 22379],
        [  390,   450, 22379,   390],
        [  450, 22379,   390,  8235],
        [22379,   390,  8235,     8],
        [  390,  8235,     8,  1658],
        [ 8235,     8,  1658,   555]])]
